In [192]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Initialiser les DataFrames
df_programmes = pd.DataFrame()
df_reunions = pd.DataFrame()
df_courses = pd.DataFrame()
df_participations = pd.DataFrame()

id_course_count = 0
id_reunion_count = 0
id_participation_count = 0

# Date du premier enregistrement 19-02-2013
# id_programme = 19022013

# Date du premier enregistrement 19-02-2013
start_date = datetime.strptime("19022013", "%d%m%Y")
end_date = datetime.strptime("19042013", "%d%m%Y")
# end_date = datetime.now()

current_date = start_date

while current_date <= end_date:
    id_programme = current_date.strftime("%d%m%Y")

    # URL avec la date de début
    url = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}"

    # Récupération du Json
    response = requests.get(url)

    if response.status_code == 200:
        json_data = response.json()

        #------------------------------------------------------------------- Construction de la table programme -------------------------------------------------------------------
        # Extraire les informations du programme
        date_programme_timestamp = json_data["programme"]["date"] // 1000
        date_programme = datetime.fromtimestamp(date_programme_timestamp).date()

        # Format personnalisé de la date en français
        date_programme_fr = date_programme.strftime("%d-%m-%Y")

        # Afficher les informations du programme
        # print(f"Id du programme : {id_programme} | Date du programme : {date_programme_fr}")
        df_programmes = pd.concat([df_programmes, pd.DataFrame([{"id_programme": id_programme, "date_programme": date_programme_fr}])], ignore_index=True)
        #------------------------------------------------------------------- Construction de la table programme -------------------------------------------------------------------

        #------------------------------------------------------------------- Construction de la table reunions --------------------------------------------------------------------

        for reunion in json_data["programme"]["reunions"]:
            # id_reunion est date + numero de la reunion exemple 19022013R1
            id_reunion_count += 1
            id_reunion = str(id_reunion_count).zfill(4)
            reunion_data = {
                "id_reunion": id_reunion,
                "id_programme": id_programme,
                "num_officiel": reunion.get("numOfficiel"),
                "nature": reunion.get("nature"),
                "code_hippodrome": reunion.get("hippodrome", {}).get("code"),
                "libelle_court_hippodrome": reunion.get("hippodrome", {}).get("libelleCourt"),
                "libelle_long_hippodrome": reunion.get("hippodrome", {}).get("libelleLong"),
                "code_pays": reunion["pays"].get("code"),
                "libelle_pays": reunion["pays"].get("libelle"),
                "audience": reunion.get("audience"),
                "statut": reunion.get("statut"),
                "disciplines_mere": reunion.get("disciplinesMere"),
                "specialites": reunion.get("specialites"),
                "meteo_nebulosite_code": reunion.get("meteo", {}).get("nebulositeCode"),
                "meteo_nebulosite_Libelle_Court": reunion.get("meteo", {}).get("nebulositeLibelleCourt"),
                "meteo_nebulosite_Libelle_Long": reunion.get("meteo", {}).get("nebulositeLibelleLong"),
                "meteo_temperature": reunion.get("meteo", {}).get("temperature"),
                "meteo_force_vent": reunion.get("meteo", {}).get("forceVent"),
                "meteo_direction_vent": reunion.get("meteo", {}).get("directionVent")
            }

            df_reunions = pd.concat([df_reunions, pd.DataFrame([reunion_data])], ignore_index=True)
            #------------------------------------------------------------------- Construction de la table reunions --------------------------------------------------------------------




            #------------------------------------------------------------------- Construction de la table courses --------------------------------------------------------------------
            num_reunion = reunion_data['num_officiel']
            url_courses = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}/R{num_reunion}"

            # Récupération du Json
            response_courses = requests.get(url_courses)

            if response_courses.status_code == 200:
                json_data_courses = response_courses.json()
            

                for course in json_data_courses["courses"]:
                    id_course_count += 1
                    id_course = str(id_course_count).zfill(6)
                    heureDepart_timestamp = course.get("heureDepart", 0) // 1000

                    # Gérer les timestamps négatifs
                    if heureDepart_timestamp < 0:
                        heureDepart = None
                    else:
                        heureDepart = datetime.fromtimestamp(heureDepart_timestamp).time()

                    # Convertir dureeCourse de millisecondes à un format lisible
                    duree_course_ms = course.get("dureeCourse", 0)
                    duree_course_minutes, duree_course_seconds = divmod(duree_course_ms // 1000, 60)
                    duree_course_formatted = f"{duree_course_minutes}m {duree_course_seconds}s"

                    # Assurez-vous que les incidents sont correctement extraits
                    incidents_list = course.get("incidents", [])
                    incidents_type = [incident.get("type") for incident in incidents_list] if isinstance(incidents_list, list) else None
                    incidents_participants = [incident.get("numeroParticipants") for incident in incidents_list] if isinstance(incidents_list, list) else None

                    specialites = reunion.get("specialites", [None, None])  # Deux spécialités ou None
                    specialite_1 = specialites[0] if len(specialites) > 0 else None
                    specialite_2 = specialites[1] if len(specialites) > 1 else None

                    course_data = {
                        "id_course": id_course,
                        "id_reunion": id_reunion,
                        "libelle": course.get("libelle"),
                        "libelle_court": course.get("libelleCourt"),
                        "heure_depart": heureDepart,
                        "parcours": course.get("parcours"),
                        "distance": course.get("distance"),
                        "distance_unit": course.get("distanceUnit"),
                        "corde": course.get("corde"),
                        "discipline": course.get("discipline"),
                        "specialite_1": specialite_1,
                        "specialite_2": specialite_2,
                        "condition_sexe": course.get("conditionSexe"),
                        "conditions": course.get("conditions"),
                        "statut": course.get("statut"),
                        "categorie_statut": course.get("categorieStatut"),
                        "duree_course": course.get("dureeCourse"),
                        "duree_course_en_minute":duree_course_formatted,
                        "montant_prix": course.get("montantPrix"),
                        "grand_prix_national_trot": course.get("grandPrixNationalTrot"),
                        "montant_total_offert": course.get("montantTotalOffert"),
                        "montant_offert_1er": course.get("montantOffert1er"),
                        "montant_offert_2eme": course.get("montantOffert2eme"),
                        "montant_offert_3eme": course.get("montantOffert3eme"),
                        "montant_offert_4eme": course.get("montantOffert4eme"),
                        "montant_offert_5eme": course.get("montantOffert5eme"),
                        "nombre_declares_partants": course.get("nombreDeclaresPartants"),
                        "incidents_type": incidents_type,
                        "incidents_participants": incidents_participants,
                        "ordre_arrivee": course.get("ordreArrivee"),
                    }
                    df_courses = pd.concat([df_courses, pd.DataFrame([course_data])], ignore_index=True)
                    #------------------------------------------------------------------- Construction de la table courses --------------------------------------------------------------------


                    #------------------------------------------------------------------- Construction de la table participations -------------------------------------------------------------
                    num_course = course.get("numOrdre")
                    url_participants = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}/R{num_reunion}/C{num_course}/participants"

                    # Récupération du Json
                    response_participants = requests.get(url_participants)

                    if response_participants.status_code == 200:
                        json_data_participants = response_participants.json()

                        for participant in json_data_participants["participants"]:
                            id_participation_count += 1
                            id_participation = str(id_participation_count).zfill(8)

                            # Convertir dureeCourse de millisecondes à un format lisible
                            temps_obtenu_ms = participant.get("tempsObtenu", 0)
                            temps_obtenu_minutes, temps_obtenu_seconds = divmod(temps_obtenu_ms // 1000, 60)
                            temps_obtenu_formatted = f"{temps_obtenu_minutes}m {temps_obtenu_seconds}s"

                            participation_data = {
                                "id_participation": id_participation,
                                "id_course": id_course,
                                "nom": participant.get("nom"),
                                "numero_cheval": participant.get("numPmu"),
                                "age": participant.get("age"),
                                "sexe": participant.get("sexe"),
                                "race": participant.get("race"),
                                "statut": participant.get("statut"),
                                "proprietaire": participant.get("proprietaire"),
                                "entraineur": participant.get("entraineur"),
                                "driver": participant.get("driver"),
                                "driverChange": participant.get("driverChange"),
                                "code_robe": participant.get("robe", {}).get("code"),
                                "libelleCourt_robe": participant.get("robe", {}).get("libelleCourt"),
                                "libelleLong_robe": participant.get("robe", {}).get("libelleLong"),
                                "nombre_courses": participant.get("nombreCourses"),
                                "nombre_victoires": participant.get("nombreVictoires"),
                                "nombre_places": participant.get("nombrePlaces"),
                                "nom_pere": participant.get("nomPere"),
                                "nom_mere": participant.get("nomMere"),
                                "ordre_arrivee": participant.get("ordreArrivee"),
                                "jument_pleine": participant.get("jumentPleine"),
                                "engagement": participant.get("engagement"),
                                "supplement": participant.get("supplement"),
                                "handicap_distance": participant.get("handicapDistance"),
                                "poids_condition_monte_change": participant.get("poidsConditionMonteChange"),
                                "temps_obtenu": participant.get("tempsObtenu"),
                                "temps_obtenu_en_minute": temps_obtenu_formatted,
                                "reduction_kilometrique": participant.get("reductionKilometrique"),
                                "allure": participant.get("allure"),
                            }
                            df_participations = pd.concat([df_participations, pd.DataFrame([participation_data])], ignore_index=True)


                    #------------------------------------------------------------------- Construction de la table participations -------------------------------------------------------------

    else:
        print(f"Échec de la récupération des données, code de statut : {response.status_code}")

    # Passer à la date suivante
    current_date += timedelta(days=1)


In [193]:
df_programmes

,id_programme,date_programme
0,19062013,19-06-2013


In [194]:
df_reunions

,id_reunion,id_programme,num_officiel,nature,code_hippodrome,libelle_court_hippodrome,libelle_long_hippodrome,code_pays,libelle_pays,audience,statut,disciplines_mere,specialites,meteo_nebulosite_code,meteo_nebulosite_Libelle_Court,meteo_nebulosite_Libelle_Long,meteo_temperature,meteo_force_vent,meteo_direction_vent
0,0001,19022013,1,DIURNE,VIN,VINCENNES,HIPPODROME DE PARIS-VINCENNES,FRA,FRANCE,NATIONAL,TERMINEE,[TROT],"[TROT_ATTELE, TROT_MONTE]",None,None,None,None,None,None
1,0002,19022013,4,NOCTURNE,CAG,CAGNES/MER,HIPPODROME DE CAGNES/MER,FRA,FRANCE,NATIONAL,TERMINEE,[TROT],[TROT_ATTELE],None,None,None,None,None,None
2,0003,19022013,5,SEMINOCTURNE,HWM,MONS (GHLIN),HIPPODROME DE MONS BELGIQUE,BEL,Belgique,NATIONAL,TERMINEE,[TROT],[TROT_ATTELE],None,None,None,None,None,None
3,0004,19022013,10,DIURNE,LIG,LIGNIERES,HIPPODROME DE LIGNIERES,FRA,FRANCE,REGIONAL,PROGRAMMEE,[TROT],[TROT_ATTELE],None,None,None,None,None,None
4,0005,20022013,3,SEMINOCTURNE,ENG,ENGHIEN,HIPPODROME D'ENGHIEN SOISY,FRA,FRANCE,NATIONAL,TERMINEE,[TROT],"[TROT_ATTELE, TROT_MONTE]",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,0768,19062013,1,DIURNE,REI,REIMS,HIPPODROME DE REIMS,FRA,FRANCE,NATIONAL,TERMINEE,[TROT],"[TROT_ATTELE, TROT_MONTE]",None,None,None,None,None,None
768,0769,19062013,2,DIURNE,COM,COMPIEGNE,HIPPODROME DE COMPIEGNE,FRA,FRANCE,NATIONAL,TERMINEE,[PLAT],[PLAT],None,None,None,None,None,None
769,0770,19062013,3,SEMINOCTURNE,VIC,VICHY,HIPPODROME DE VICHY,FRA,FRANCE,NATIONAL,TERMINEE,[TROT],"[TROT_MONTE, TROT_ATTELE]",None,None,None,None,None,None
770,0771,19062013,4,DIURNE,ASC,ASCOT,HIPPODROME D'ASCOT GB,GBR,Royaume-Uni,NATIONAL,TERMINEE,[PLAT],[PLAT],None,None,None,None,None,None


In [195]:
print(df_courses)

     id_course id_reunion                  libelle    libelle_court  \
0       000001       0001          PRIX DU CHESNAY       DU CHESNAY   
1       000002       0001         PRIX DE CHALLANS      DE CHALLANS   
2       000003       0001             PRIX DE LENS          DE LENS   
3       000004       0001          PRIX DE CARHAIX       DE CARHAIX   
4       000005       0001           PRIX DE CERISY        DE CERISY   
...        ...        ...                      ...              ...   
5056    005057       0772         PRIX DE MANOSQUE      DE MANOSQUE   
5057    005058       0772        PRIX DE MONTREDON        MONTREDON   
5058    005059       0772     PRIX DU HARAS DE PAU     HARAS DE PAU   
5059    005060       0772  PRIX DE SAINT HIPPOLYTE  SAINT HIPPOLYTE   
5060    005061       0772           PRIX D'ENGHIEN        D'ENGHIEN   

     heure_depart parcours  distance distance_unit         corde discipline  \
0        13:50:00               2850         METRE  CORDE_GAUCHE    

In [196]:
df_participations

,id_participation,id_course,nom,numero_cheval,age,sexe,race,statut,proprietaire,entraineur,...,ordre_arrivee,jument_pleine,engagement,supplement,handicap_distance,poids_condition_monte_change,temps_obtenu,temps_obtenu_en_minute,reduction_kilometrique,allure
0,00000001,000001,RODGERS,1,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,Ecurie PELHEM,M. IZAAC,...,9,False,False,0,2850,False,213700,3m 33s,710000,TROT
1,00000002,000001,RAVANELLO,2,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,CH. NEIRINCK,V. MARTENS,...,None,False,False,0,2850,False,None,0m 0s,None,TROT
2,00000003,000001,SERGENT DU RIB,3,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,Ecurie RIB,J.L.CL. DERSOIR,...,4,False,False,0,2850,False,213110,3m 33s,708000,TROT
3,00000004,000001,SEGUINEL MABON,4,7,MALES,TROTTEUR FRANCAIS,PARTANT,Y. DREUX,Y. DREUX,...,1,False,False,0,2850,False,211830,3m 31s,703000,TROT
4,00000005,000001,SILVER DES PRES,5,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,B. HELIAS,F. SOULOY,...,2,False,False,0,2850,False,212470,3m 32s,706000,TROT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63262,00063263,005061,SHOWTIME,8,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,Ecurie Sylvain ROUBAUD,Y.A. BRIAND,...,2,False,False,0,2300,False,174260,2m 54s,75800,TROT
63263,00063264,005061,TORNADO DE BLEMONT,9,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,AL. GALLO,D. ALEXANDRE,...,11,False,False,0,2300,False,181900,3m 1s,79100,TROT
63264,00063265,005061,TEDDY BRADOR,10,6,MALES,TROTTEUR FRANCAIS,PARTANT,Ecurie DARCHE,N. ENSCH,...,10,False,False,0,2300,False,180450,3m 0s,78500,TROT
63265,00063266,005061,ROC D'ARCY,11,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,Mlle S. OYER,S. CINGLAND,...,8,False,False,0,2300,False,178330,2m 58s,77500,TROT
